In [3]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [4]:
import numpy as np
from chainconsumer import ChainConsumer
#from corner import corner

In [5]:
! ls -lt /scratch/users/swmclau2/PearceMCMC/*chain*cosmo*.npy

-rw-r--r-- 1 swmclau2 kipac 13800000063 Nov 10 05:08 /scratch/users/swmclau2/PearceMCMC/1000_walkers_50000_steps_chain_cosmo_zheng_xi_lowmsat.npy
-rw-r--r-- 1 swmclau2 kipac  2338824063 Nov  8 16:33 /scratch/users/swmclau2/PearceMCMC/500_walkers_10000_steps_chain_cosmo_zheng_xi_lowmsat.npy
-rw-r--r-- 1 swmclau2 kipac    40443999 Nov  8 11:26 /scratch/users/swmclau2/PearceMCMC/26_walkers_10000_steps_chain_cosmo_zheng_xi_lowmsat.npy
-rw-r--r-- 1 swmclau2 kipac          63 Nov  8 08:59 /scratch/users/swmclau2/PearceMCMC/25_walkers_10000_steps_chain_cosmo_zheng_xi_lowmsat.npy


In [6]:
! du /u/ki/swmclau2/des/PearceMCMC/*chain*cosmo*.npy -sh

du: cannot access ‘/u/ki/swmclau2/des/PearceMCMC/*chain*cosmo*.npy’: No such file or directory


In [7]:
#fname = 'ab_sham_chain.npy'
#fname = '/u/ki/swmclau2/des/PearceMCMC/chain.npy'
#fname = '/u/ki/swmclau2/des/PearceMCMC/500_walkers_10000_steps_chain_cosmo_zheng_xi_lowmsat_fixedCosmo.npy'
fname = '/scratch/users/swmclau2/PearceMCMC/1000_walkers_50000_steps_chain_cosmo_zheng_xi_lowmsat.npy'

In [8]:
n_walkers = 1000
n_burn = 40000

In [10]:
from itertools import islice

In [28]:
chain_list = []
with open(fname, 'r') as f:
    step_counter = 0
    print f.readline() #header
    while True:
        next_lines = islice(f, n_walkers)
        step_counter+=1
        
        if step_counter % 1000 == 0:
            print step_counter, len(chain_list)
        
        if step_counter < n_burn:
            for line in next_lines:
                pass
            continue

        next_lines = np.array([ np.fromstring(line, sep = ' ') for line in next_lines], dtype=float)

        if len(next_lines) < 1:
            break
            
        chain_list.append(next_lines)

#Neff	logM0	logM1	H0	w0	ln10As	omch2	sigma_logM	alpha	ns	ombh2

1000 0
2000 0
3000 0
4000 0
5000 0
6000 0
7000 0
8000 0
9000 0
10000 0
11000 0
12000 0
13000 0
14000 0
15000 0
16000 0
17000 0
18000 0
19000 0
20000 0
21000 0
22000 0
23000 0
24000 0
25000 0
26000 0
27000 0
28000 0
29000 0
30000 0
31000 0
32000 0
33000 0
34000 0
35000 0
36000 0
37000 0
38000 0
39000 0
40000 0
41000 1000
42000 2000
43000 3000
44000 4000
45000 5000
46000 6000
47000 7000
48000 8000
49000 9000
50000 10000


In [33]:
chain = np.vstack(chain_list)

In [35]:
chain.shape

(10001000, 11)

In [36]:
#n_walkers = 1000
n_params = chain.shape[1] if len(chain.shape) > 1 else 1

In [37]:
MAP = chain.mean(axis = 0)
print MAP

[  2.96838130e+00   1.41509166e+01   1.35680742e+01   6.98327127e+01
  -1.08524307e+00   3.11005712e+00   1.19180235e-01   1.87935995e-01
   9.89691819e-01   9.66215217e-01   2.21522730e-02]


In [38]:
param_names = [r'$N_{eff}$', r'$\log(M_0)$',r'$\log(M_1)$', r'$H_0$',r'$w_0$', r'$\ln(10A_s)$',  r'$\Omega_c h^2$',
               r'$\sigma_{\log M }$', r'$\alpha$',  r'$n_s$', r'$\Omega_b h^2$']

In [39]:
hod_param_names = [r'$\sigma_{\log M }$', r'$\log(M_0)$',r'$\log(M_1)$',r'$\alpha$' ]
cosmo_param_names = [r'$N_{eff}$', r'$H_0$', r'$w_0$', r'$\Omega_c h^2$',r'$\ln(10A_s)$',r'$n_s$',r'$\Omega_b h^2$' ]

In [40]:
chain.shape

(10001000, 11)

In [41]:
#fixed some params
#param_names = [r'$\log{M_{min}}$',r'$\sigma_{log{M}}$',r'$\log{M_1}$', r'$\alpha$']#, r'$\log{M_0}$', r'$\log{M_1}$', r'$\alpha$']

In [42]:
c = ChainConsumer()
c.add_chain(chain, walkers=n_walkers, parameters = param_names)
c.configure(statistics='cumulative')

In [43]:
#true_vals = np.array([12.9390382,12.51024343,  14.36144524,   0.73766725,  1.06822384])
#true_vals = np.array([12.9390382,0.0, 12.51024343,  14.36144524, 0.0,  0.73766725,  1.06822384])

In [44]:
from pearce.mocks import cat_dict
cosmo_params = {'simname':'testbox', 'boxno': 3, 'realization':0, 'scale_factors':[1.0], 'system': 'sherlock'}
cat = cat_dict[cosmo_params['simname']](**cosmo_params)#construct the specified catalog!

In [45]:
cpv = cat._get_cosmo_param_names_vals()

print  {key: val for key, val in zip(cpv[0], cpv[1])}

{'Neff': 3.7000000000000002, 'H0': 70.731700000000004, 'w0': -1.13151, 'omch2': 0.12282999999999999, 'ln10As': 3.11395, 'ns': 0.953515, 'ombh2': 0.021762900000000002}


In [46]:
cosmo_true_vals = [3.7,70.7317,-1.13151, 0.12283,3.11395, 0.953515,  0.021762]

In [47]:
emulation_point = [('logM0', 14.0), ('sigma_logM', 0.2),
                    ('alpha', 1.083),('logM1', 13.7)]

In [48]:
#hod_true_vals = [0.2, 14.0, 13.7, 1.083]
hod_true_vals = [13.7, 14.0, 0.2, 1.083]

In [49]:
fig = c.plotter.plot(figsize=(8,8))#, parameters = hod_param_names, truth = hod_true_vals)# parameters = [param_names[i] for i in (1, 4)])
                    #, truth = true_vals)
fig.show()

RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 



RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 



In [ ]:
fig = c.plotter.plot(figsize=(10,10), parameters = cosmo_param_names, truth = cosmo_true_vals)# parameters = [param_names[i] for i in (1, 4)])
                    #, truth = true_vals)
fig.show()

In [ ]:
#c.configure(statistics='max')
fig = c.plotter.plot_distributions(figsize=(10, 3) ,parameters = hod_param_names, truth = hod_true_vals)
fig.show()

In [ ]:
#c.configure(statistics='max')
fig = c.plotter.plot_distributions(figsize=(10, 6) , parameters = cosmo_param_names, truth = cosmo_true_vals)
fig.show()

In [ ]:
gelman_rubin_converged = c.diagnostic.gelman_rubin()
print gelman_rubin_converged

In [ ]:
np.sqrt(np.diag(c.analysis.get_covariance()[1]))

In [ ]:
summary  = c.analysis.get_summary()
for key, val in summary.iteritems():
    print key, val[1]

In [ ]:
MAP = np.array([summary[p][1] for p in param_names])
print MAP

In [ ]:
print param_names